# DETECTEUR DE PITCH TEMPS REEL

In [5]:
import pyaudio
import struct
import matplotlib.pyplot as plt
import numpy as np
from scipy.fftpack import fft
from scipy.fftpack import rfft

from scipy import signal
import os
%matplotlib tk


In [18]:

CHUNK = 1024 * 2 #longeur du tronçons de donnée à analyser, plus grand plus fft precise
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100   #frequence d'échantillonage
SEUIL = 0.9 #pour afficher le seuil, 1 est le max 
#PAS = 1024 * 4 # troncons que traite effectivemnt la fft avec nfft


alpha = 2**(1/12) #pas pour la gamme tempérée
note_grave = 110
Nb_Octave = 2
OCTAVE = np.zeros(13*Nb_Octave)
for i in range (13*Nb_Octave):
    OCTAVE[i]= note_grave*(alpha**i)

    
Freq_Min =note_grave #affichage de la freq min pour le temps-freq
Freq_Max = note_grave*(2**Nb_Octave)  #affichage de la freq max pour le temps-freq

freq_min_recherche = 110
freq_max_recherche = 440

TimeListen = 60 #largeur aproximatif d'écoute en s de l'analyse temps-freq

In [19]:
def calcul_periodogramme(x,Fe,duree_sous_bloc):
    ### périodogramme moyenné. Sections de 0.1 seconde
    N=len(x)
    N1=int(np.floor(duree_sous_bloc*Fe))
    K=int(np.floor(N/N1))
    w=signal.hamming(N1)
    ind1 = 0
    ind2 = N1-1
    b = w*x[ind1:(ind2+1)] 
    f, B = signal.periodogram(b, Fe,nfft =44100)
    B=B/N1
    while ind2+N1<N:
        ind1=ind1+N1
        ind2=ind2+N1
        b=w*x[ind1:(ind2+1)]
        f, aux = signal.periodogram(b, Fe, nfft =44100)
        B=B+aux/N1
    B=B/K
    #B=10*np.log10(B)
    return(f,B)
    

In [20]:

p = pyaudio.PyAudio()

#data du micro
stream = p.open(
    format = FORMAT ,
    channels = CHANNELS,
    rate = RATE,
    input= True,
    output = True,
    frames_per_buffer=CHUNK)



fig, (ax,ax_fft,ax2) = plt.subplots(3,figsize=(10,7))



ax.set_xlim(0,CHUNK)
ax2.set_xlim(0,TimeListen*7.5)
ax_fft.set_xlim(0,RATE/20)


#variable pour plotter
x = np.arange(0, 1 * CHUNK ,1)
x2 = np.arange(0,1*CHUNK,1)
x_fft, aux = calcul_periodogramme(np.zeros(CHUNK),RATE,0.01)


line, = ax.plot(x,np.random.rand(CHUNK),'-',lw=2)
line_x2, = ax2.plot(x2,np.random.rand(CHUNK),'-',lw=2)
line_x_fft, = ax_fft.plot((x_fft),np.random.rand(len(x_fft)),'-',lw=2)
#plt.setp(ax_fft,xticks=[55,110,220,440,880,1760,3560])


ax.set_title("AUDIO FORME D'ONDE")

ax.set_ylabel('volume')
ax.set_ylim(-1,1)
#ax.set_xlim(0,CHUNK)


ax_fft.set_title("AUDIO SPECTRE")
ax_fft.set_ylabel('Amplitude')
ax_fft.set_ylim(0,1)
#ax_fft.set_xlim(0.001,CHUNK)


ax2.set_title("ANALYSE FREQ/TEMPS")
ax2.set_ylabel('freq')
plt.yticks(OCTAVE, ["La", "Sib", "Si","Do","Do#","Ré","Mib","Mi","Fa","Fa#","Sol","Sol#","La" ]*Nb_Octave)


plt.grid()
ax2.set_ylim(Freq_Min,Freq_Max)


m = np.zeros(CHUNK)
i = 0

while True:
    data = stream.read(CHUNK)#data en binaire
    data_int = np.array(struct.unpack(str(2*CHUNK)+ 'B', data),dtype='b')[::2] /128
    line.set_ydata(data_int)
    
    if(np.max(data_int)>SEUIL):
        print("depasse le seuil")
    
    freq , periodo = calcul_periodogramme(data_int,RATE,0.01)
    line_x_fft.set_ydata(10000000*periodo) 
    
    
    cond = (freq >freq_min_recherche) * (freq <freq_max_recherche)
    periodo2 = periodo*cond
    ma = np.argmax(periodo2)
    pitch = freq[ma]
    m[i]= ma
    print(m[i])
    i= i+1
    line_x2.set_ydata(m)
    ax2.set_xlim(left = max (0, i-50), right = i+20)
    
    fig.canvas.draw()
    fig.canvas.flush_events()



112.0
111.0
114.0
111.0
115.0
131.0
117.0
114.0
113.0
113.0
112.0
112.0
113.0
133.0
111.0
113.0
112.0
111.0
113.0
112.0
112.0
112.0
289.0
depasse le seuil
290.0
depasse le seuil
290.0
292.0
299.0
296.0
295.0
295.0
296.0
296.0
294.0
264.0
262.0
260.0
260.0
261.0
263.0
263.0
263.0
263.0
243.0
232.0
231.0
233.0
233.0
233.0
232.0
223.0
depasse le seuil
221.0
depasse le seuil
221.0
depasse le seuil
219.0
depasse le seuil
220.0
depasse le seuil
223.0
depasse le seuil
219.0
depasse le seuil
203.0
depasse le seuil
195.0
depasse le seuil
196.0
depasse le seuil
196.0
depasse le seuil
198.0
depasse le seuil
203.0
depasse le seuil
201.0
depasse le seuil
198.0
175.0
170.0
170.0
172.0
167.0
169.0
173.0
depasse le seuil
187.0
depasse le seuil
193.0
depasse le seuil
192.0
depasse le seuil
193.0
depasse le seuil
201.0
depasse le seuil
197.0
depasse le seuil
189.0
depasse le seuil
199.0
depasse le seuil
209.0
depasse le seuil
203.0
depasse le seuil
202.0
depasse le seuil
230.0
230.0
232.0
233.0
234.0
23

TclError: invalid command name "pyimage53"